In [3]:
import synapseclient
from synapseclient import File
import synapseutils
import pandas as pd
from IPython.display import display

syn = synapseclient.login()
PROJECTID = 'syn8316993'

Welcome, Larsson Omberg!



/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Find the files that were uploaded with a query

In [4]:
files = synapseutils.query2df(syn.chunkedQuery('select name from file where parentId=="syn8317652"'))
files = map(syn.get, files.id)

# Read in the data

In [5]:
dfs = [pd.read_csv(f.path) for f in files]
display(dfs[0].head())
display(dfs[1].head())
display(dfs[3].head())

,vegetables,meat,dairy,wine
0,2,3,1,1
1,2,2,3,3
2,1,3,4,2
3,4,2,0,1
4,2,1,2,3


,vegetables,meat,dairy,beer
0,2,6,2,1
1,1,2,1,1
2,2,3,0,0
3,3,5,1,3
4,0,4,2,1


,vegetables,meat,dairy,beer,wine
0,4,4,1,0,1
1,2,5,1,1,1
2,2,3,4,2,0
3,0,3,3,0,1
4,1,4,4,2,0


# Convert consumption

In [6]:
name_translation = {'sweden.csv': 'Sweden', 
                    'germany.csv': 'Germany',
                    'france.csv': 'France',
                    'usa.csv': 'United States'}

merged = []
for f, df in zip(files, dfs):
    cols = list(set(['beer', 'wine', 'vokda']) & set(df.columns))
    df['alcohol'] = df[cols].sum(1)
    df['country'] = name_translation[f.name]
    df.drop(cols, axis=1, inplace=True)
    display(df.head())
    merged.append(df)

df = pd.concat(merged)

,vegetables,meat,dairy,alcohol,country
0,2,3,1,1,France
1,2,2,3,3,France
2,1,3,4,2,France
3,4,2,0,1,France
4,2,1,2,3,France


,vegetables,meat,dairy,alcohol,country
0,2,6,2,1,Germany
1,1,2,1,1,Germany
2,2,3,0,0,Germany
3,3,5,1,3,Germany
4,0,4,2,1,Germany


,vegetables,meat,dairy,alcohol,country
0,1,1,0,1,Sweden
1,4,2,2,1,Sweden
2,2,1,3,0,Sweden
3,0,3,0,0,Sweden
4,1,1,2,2,Sweden


,vegetables,meat,dairy,alcohol,country
0,4,4,1,1,United States
1,2,5,1,2,United States
2,2,3,4,2,United States
3,0,3,3,1,United States
4,1,4,4,2,United States


### Push the content back to Synapse

In [7]:
source = 'https://github.com/larssono/SynapseDemo/blob/master/Curate%20Country%20Example.ipynb'
df.to_csv('merged_alcohol_consumption.csv', index=False)
outfile = File('merged_alcohol_consumption.csv', parentId=PROJECTID)
outfile = syn.store(outfile, used=files, executed=source)


##################################################
 Uploading file to Synapse storage 
##################################################


In [ ]:
syn.onweb(outfile)